In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
data = {
    'distance_matrix': [
        [0, 10, 20, 30, 40],
        [10, 0, 50, 60, 70],
        [20, 50, 0, 40, 30],
        [30, 60, 40, 0, 20],
        [40, 70, 30, 20, 0]
    ],
    'pickups_deliveries': [
        [1, 2],
        [2, 4],
        [3, 5]
    ],
    'num_vehicles': 2,
    'depot': 0
}

In [4]:
manager = pywrapcp.RoutingIndexManager(5, 2, 0)
routing = pywrapcp.RoutingModel(manager)

In [5]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [6]:
transient_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transient_callback_index)

In [7]:
dimension_name = 'Distance'
routing.AddDimension(transient_callback_index,
                     0,
                     3000,
                     True,
                     dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

In [ ]:
for request in data['pickups_deliveries']:

    pickup_index = manager.NodeToIndex(request[0])
    delivery_index = manager.NodeToIndex(request[1])
    routing.AddPickupAndDelivery(pickup_index, delivery_index)
    routing.solver().Add(
        routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
    routing.solver().Add(
        distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))